In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.
C:\Users\abdi\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\abdi\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\abdi\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\abdi\Anaconda3\envs\tensor

In [16]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "eng2por.txt"

In [17]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [18]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 85
Max sequence length for inputs: 16
Max sequence length for outputs: 44


In [19]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [20]:
decoder_input_data.shape

(10000, 44, 85)

In [30]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [31]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, None, 69)     0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, None, 85)     0                                            
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, 256), (None, 333824      input_13[0][0]                   
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, None, 256),  350208      input_14[0][0]                   
                                                                 lstm_9[0][1]               

In [32]:
print("encoder_input_data shape:",encoder_input_data.shape)
print("decoder_input_data shape:",decoder_input_data.shape)
print("decoder_target_data shape:",decoder_target_data.shape)

encoder_input_data shape: (10000, 16, 69)
decoder_input_data shape: (10000, 44, 85)
decoder_target_data shape: (10000, 44, 85)


In [59]:
# Run training
from keras.optimizers import * 

model.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy',metrics = ['accuracy'])
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = batch_size,
          epochs = 100, 
          validation_split = 0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 12s 1ms/step - loss: 0.2237 - acc: 0.2975 - val_loss: 0.6398 - val_acc: 0.2816
Epoch 2/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.1760 - acc: 0.3100 - val_loss: 0.6178 - val_acc: 0.2843
Epoch 3/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.1514 - acc: 0.3176 - val_loss: 0.6144 - val_acc: 0.2869
Epoch 4/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.1357 - acc: 0.3227 - val_loss: 0.6189 - val_acc: 0.2871
Epoch 5/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.1244 - acc: 0.3264 - val_loss: 0.6272 - val_acc: 0.2875
Epoch 6/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.1133 - acc: 0.3305 - val_loss: 0.6324 - val_acc: 0.2873
Epoch 7/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.1042 - acc: 0.3336 - val_loss: 0.6452 - val_acc: 0.2871


In [60]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [61]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [62]:
def encoding_sentence (input_text):
    
    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    
    #for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[1, t, input_token_index[char]] = 1.
    return encoder_input_data[1:2]

In [80]:
sentence = "go away"
input_seq = encoding_sentence(sentence)
print(input_seq.shape)
print(input_seq)

decoded_sentence = decode_sequence(input_seq)
print('-')
print('Input sentence:', sentence)
print('Decoded sentence:', decoded_sentence)

(1, 16, 69)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
-
Input sentence: go away
Decoded sentence: Que bas!



In [43]:
input_seq.shape

(1, 16, 69)

In [63]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Vá.

-
Input sentence: Go.
Decoded sentence: Vá.

-
Input sentence: Hi.
Decoded sentence: Oi.

-
Input sentence: Run!
Decoded sentence: Corram!

-
Input sentence: Run!
Decoded sentence: Corram!

-
Input sentence: Run!
Decoded sentence: Corram!

-
Input sentence: Run.
Decoded sentence: Corram!

-
Input sentence: Run.
Decoded sentence: Corram!

-
Input sentence: Run.
Decoded sentence: Corram!

-
Input sentence: Who?
Decoded sentence: Quem?

-
Input sentence: Wow!
Decoded sentence: Uau!

-
Input sentence: Wow!
Decoded sentence: Uau!

-
Input sentence: Wow!
Decoded sentence: Uau!

-
Input sentence: Fire!
Decoded sentence: Fogo!

-
Input sentence: Help!
Decoded sentence: Ajuda!

-
Input sentence: Help!
Decoded sentence: Ajuda!

-
Input sentence: Jump!
Decoded sentence: Pule!

-
Input sentence: Jump.
Decoded sentence: Pule.

-
Input sentence: Jump.
Decoded sentence: Pule.

-
Input sentence: Stop!
Decoded sentence: Parem!

-
Input sentence: Stop!
Decode

In [ ]:
#from keras.models import load_model
model.save('LSTM_Seq2Seq_translator.h5')
#model = load_model('LSTM_Seq2Seq_translator.h5')